In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib   
from scipy.spatial.distance import pdist, squareform
from io import StringIO

matplotlib.rcParams['pdf.fonttype'] = 42

strain = 0.30

In [2]:
def data_org(file_path):
    buffer = StringIO()
    current_time = None

    with open(file_path, "r") as f:
        for line in f:
            if line.strip() == "ITEM: TIMESTEP":
                current_time = f.readline().strip()
                continue
                
            parts = line.strip().split()
            if len(parts) == 5:
                buffer.write(f"{current_time} {' '.join(parts)}\n")

    buffer.seek(0)
    df = pd.read_csv(buffer, sep=' ', 
                        names=["time", "id", "type", "x", "y", "z"])
    return df

In [3]:
def contact_index(lamins_df, chromatin_df, time):

    last_chromatin = chromatin_df.loc[chromatin_df["time"] == time].reset_index(drop=True)
    last_lamins = lamins_df.loc[lamins_df["time"] == time].reset_index(drop=True)
    
    chromatin_coords = last_chromatin[["x", "y", "z"]].to_numpy()
    lamins_coords = last_lamins[["x", "y", "z"]].to_numpy()

    contact_distances = []

    for chromatin in chromatin_coords:

        distances = np.sqrt(np.sum((chromatin - lamins_coords)**2, axis=1))
        contact_distances.append(np.min(distances))
    
    last_chromatin["contact index"] = contact_distances
    
    return last_chromatin

In [4]:
import warnings

warnings.filterwarnings('ignore')

def calculate_contacts(lamins_df,chromatin_df, time):

    last_chromatin_df = chromatin_df[chromatin_df["time"] == time]

    last_chromatin_df = last_chromatin_df.sort_values(by = "id")

    last_chromatin_df.reset_index(drop = True, inplace = True)

    last_lamins_df = lamins_df[lamins_df["time"] == time]

    last_lamins_df = last_lamins_df.sort_values(by = "id")

    last_lamins_df.reset_index(drop = True, inplace = True)

    lamin_map = np.array([])
    lamin_non_map = np.array([])
    
    for ind, lam in last_lamins_df.iterrows():
        distance = (last_chromatin_df["x"] - lam["x"])**2 + (last_chromatin_df["y"] - lam["y"])**2 + (last_chromatin_df["z"] - lam["z"])**2
        distance = distance**0.5
        lamin_map = np.append(lamin_map, len(distance[distance <= 2.5]))
        lamin_non_map = np.append(lamin_non_map, len(distance[distance > 2.5]))
        
    last_lamins_df["chromatin contacts"] = lamin_map

    last_lamins_df["non chromatin contacts"] = lamin_non_map

    return last_lamins_df

In [5]:
def plot_contacts(last_lamins_df):

    scaler = MinMaxScaler()

    last_lamins_df['norm. chromatin contacts'] = scaler.fit_transform(last_lamins_df['chromatin contacts'].values.reshape(-1, 1))
    last_lamins_df['norm. non chromatin contacts'] = scaler.fit_transform(last_lamins_df['non chromatin contacts'].values.reshape(-1, 1))

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    sns.kdeplot(data=last_lamins_df, x='norm. chromatin contacts', fill=True, alpha=0.5, label='chromatin contacts', ax=ax)
    sns.kdeplot(data=last_lamins_df, x='norm. non chromatin contacts', fill=True, alpha=0.5, label='non chromatin contacts', ax=ax)
    ax.set_xlabel('Normalized')
    ax.set_ylabel('Density')
    ax.set_title('Contacts')
    ax.legend(loc='upper right')

    plt.show()

In [6]:
def ps_segment(chr_ind_last):

        het_mask = chr_ind_last['type'].isin([2, 8]).values
        euc_mask = chr_ind_last['type'].isin([3, 7]).values

        k_step = 4
        k_s = np.unique(np.round([k_step * (1.11785**i) for i in range(65)]).astype(int))
        max_k = k_s.max()

        chr_length = 6002
        num_chr = 8

        results = {'het': np.zeros_like(k_s, dtype=float), 
                'euc': np.zeros_like(k_s, dtype=float),
                'het_norm_by_count': np.zeros_like(k_s, dtype=float),
                'euc_norm_by_count': np.zeros_like(k_s, dtype=float)}
        
        counts = {'het': np.zeros_like(k_s, dtype=float),
                'euc': np.zeros_like(k_s, dtype=float),
                'het_norm_by_count': np.zeros_like(k_s, dtype=float),
                'euc_norm_by_count': np.zeros_like(k_s, dtype=float)}

        for chr_n in range(num_chr):

                chr_data = chr_ind_last.iloc[chr_n*chr_length : (chr_n+1)*chr_length]
                coords = chr_data[['x', 'y', 'z']].values
                types = chr_data['type'].values

                dist_matrix = squareform(pdist(coords))

                for name, type_mask, target_types in [('het', het_mask, [2, 8]),
                                                        ('euc', euc_mask, [3, 7])]:

                        target_idx = np.where(type_mask[chr_n*chr_length : (chr_n+1)*chr_length])[0]
                        
                        for i in target_idx:

                                valid_contacts = (dist_matrix[i] <= 2.5) & (np.isin(types, target_types))
                                
                                cum_contacts = np.cumsum(valid_contacts)
                                
                                for k_idx, k in enumerate(k_s):
                                        
                                        if i + k >= chr_length - 4:
                                                continue
                                        
                                        contacts = cum_contacts[i+k] - cum_contacts[i]
                                        results[name][k_idx] += contacts/ k
                                        counts[name][k_idx] += 1

                results['het'] /= np.where(het_mask)[0].shape[0]
                results['euc'] /= np.where(euc_mask)[0].shape[0]

                results['het_norm_by_count'] = results['het'] / counts['het']
                results['euc_norm_by_count'] = results['euc'] / counts['euc']

        return results, k_s


In [7]:
def process_df(df):

    is_lamin = df["type"].isin([1, 9, 10])
    coords = df[["x", "y", "z"]].values
    
    radial = np.linalg.norm(coords, axis=1)
    
    lamin_df = df[is_lamin].copy()
    chromatin_df = df[~is_lamin].copy()
    
    lamin_df["lamin radial_distance"] = radial[is_lamin]
    chromatin_df["chromatin radial_distance"] = radial[~is_lamin]
    
    return (
        lamin_df.sort_values("id", ignore_index=True),
        chromatin_df.sort_values("id", ignore_index=True)
    )

def strain_fc(lamins_df):

    max_y_initial = lamins_df.loc[lamins_df['time'] == 0, 'y'].max()
    
    max_y_per_time = lamins_df.groupby('time', sort=True)['y'].max().reset_index()
    
    max_y_per_time['strain'] = (max_y_per_time['y'] - max_y_initial) / max_y_initial
    
    return max_y_per_time[['time', 'strain']]

In [8]:
from tqdm import tqdm
import os 

def process_pulling_data(base_path, condition, output_path, strain):

    ps_segment_data_last_array_het = np.zeros((63, 111))
    ps_segment_data_first_array_het = np.zeros((63, 11))

    ps_segment_data_last_array_euc = np.zeros((63, 111))
    ps_segment_data_first_array_euc = np.zeros((63, 11))

    config = 1

    for rep in tqdm(range(1, 11)):
        
        path = f'{base_path}/r{rep}/dump.pulling'

        df = data_org(path)

        lamins_df, chromatin_df = process_df(df)

        time_strain = strain_fc(lamins_df)

        time_strain.sort_values(by = 'time', inplace = True)

        closest_time = time_strain[(time_strain['strain'] > strain - 0.01) & (time_strain['strain'] < strain + 0.01)].iloc[0]['time']

        closes_time_list = [closest_time - 5000, closest_time - 4000, closest_time - 3000, closest_time - 2000, closest_time - 1000, closest_time, closest_time + 1000, 
                            closest_time + 2000, closest_time + 3000, closest_time + 4000, closest_time + 5000]

        print(strain, closest_time)

        chr_ind_0_FC = contact_index(lamins_df, chromatin_df, 0)   

        PSFS_FTNC_seg = ps_segment(chr_ind_0_FC)

        ps_segment_data_first_array_het[:,rep] = PSFS_FTNC_seg[0]['het']
        ps_segment_data_first_array_euc[:,rep] = PSFS_FTNC_seg[0]['euc']
        
        ps_segment_data_first_array_het[:,0] = PSFS_FTNC_seg[1]
        ps_segment_data_first_array_euc[:,0] = PSFS_FTNC_seg[1]

        ps_segment_data_last_array_het[:,0] = PSFS_FTNC_seg[1]      
        ps_segment_data_last_array_euc[:,0] = PSFS_FTNC_seg[1]
        
        for ind, time in (enumerate(closes_time_list)):

            chr_ind_last_FC = contact_index(lamins_df, chromatin_df, closest_time)   

            PSLS_FTNC_seg = ps_segment(chr_ind_last_FC)
            
            ps_segment_data_last_array_het[:,config] = PSLS_FTNC_seg[0]['het']
            ps_segment_data_last_array_euc[:,config] = PSLS_FTNC_seg[0]['euc']

            config += 1

            print(ind, time)
            
    pd.DataFrame(ps_segment_data_first_array_het).to_csv(f'{output_path}/ps_{condition}_{rep}_{strain}_het_first.csv', index = False)
    pd.DataFrame(ps_segment_data_last_array_het).to_csv(f'{output_path}/ps_{condition}_{rep}_{strain}_het_last.csv', index = False)

    pd.DataFrame(ps_segment_data_first_array_euc).to_csv(f'{output_path}/ps_{condition}_{rep}_{strain}_euc_first.csv', index = False)
    pd.DataFrame(ps_segment_data_last_array_euc).to_csv(f'{output_path}/ps_{condition}_{rep}_{strain}_euc_last.csv', index = False)

    return ps_segment_data_first_array_het, ps_segment_data_last_array_het, ps_segment_data_first_array_euc, ps_segment_data_last_array_euc


In [14]:
def process_and_plot_ps_data(ps_data_first, ps_data_last, output_path, chromatin_type, name, b_color, color, strain):

    columns_last = ['Genomic Distance'] + [f'P(s)_{i}' for i in range(1, 111)]

    columns_first = ['Genomic Distance'] + [f'P(s)_{i}' for i in range(1, 11)]

    PSFS_FTNC_s = pd.DataFrame(ps_data_first, columns=columns_first)

    PSLS_FTNC_s = pd.DataFrame(ps_data_last, columns=columns_last)

    
    for df in [PSFS_FTNC_s, PSLS_FTNC_s]:

        df['mean'] = df.iloc[:, 1:].mean(axis=1)
        df['sem'] = df.iloc[:, 1:-1].sem(axis=1)

    PSFS_FTNC_melt_s = PSFS_FTNC_s.melt(id_vars=['Genomic Distance', 'mean', 'sem'], var_name='rep', value_name='P(s)')
    PSLS_FTNC_melt_s = PSLS_FTNC_s.melt(id_vars=['Genomic Distance', 'mean', 'sem'], var_name='rep', value_name='P(s)')

    fig, axes = plt.subplots(1, 1, figsize=(3, 3), dpi=300)

    PSFS_FTNC_melt_s_FILT_plot = PSFS_FTNC_melt_s[
    (PSFS_FTNC_melt_s['P(s)'] >= 0.00001)]

    PSLS_FTNC_melt_s_FILT_plot = PSLS_FTNC_melt_s[
    (PSLS_FTNC_melt_s['P(s)'] >= 0.00001)]

    sns.lineplot(data=PSFS_FTNC_melt_s_FILT_plot, x='Genomic Distance', y='P(s)', errorbar="se", color=b_color, linewidth=1)
    sns.lineplot(data=PSLS_FTNC_melt_s_FILT_plot, x='Genomic Distance', y='P(s)', errorbar="se", linewidth=1, color=color)    

    axes.set_xscale('log')
    axes.set_yscale('log')
    axes.set_xlim((0, 6002))
    axes.set_title(f'P(s) vs Genomic Distance, {chromatin_type}')

    plt.tight_layout()
    plt.savefig(f'{output_path}/PS_Segment_{name}_{chromatin_type}_{strain}_reps.pdf', bbox_inches='tight')
    plt.close()

    return PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s


In [15]:
def filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, chromatin_type, condition, b_color, color, strain):

    PSFS_FTNC_melt_s_filtered = PSFS_FTNC_melt_s[
        (PSFS_FTNC_melt_s['Genomic Distance'] >= 50) & 
        (PSFS_FTNC_melt_s['Genomic Distance'] <= 1400) &
        (PSFS_FTNC_melt_s['P(s)'] >= 0.0005) &
        (PSFS_FTNC_melt_s['P(s)'] <= 0.015)
    ]

    PSLS_FTNC_melt_s_filtered = PSLS_FTNC_melt_s[
        (PSLS_FTNC_melt_s['Genomic Distance'] >= 50) & 
        (PSLS_FTNC_melt_s['Genomic Distance'] <= 1400) &
        (PSLS_FTNC_melt_s['P(s)'] >= 0.0005) &
        (PSLS_FTNC_melt_s['P(s)'] <= 0.015)
    ]

    fig, axes = plt.subplots(1, 1, figsize=(4, 4), dpi=300)

    sns.lineplot(data=PSFS_FTNC_melt_s_filtered, x='Genomic Distance', y='P(s)', 
                 errorbar="se", color=b_color, linewidth=1, err_kws={'alpha': 0.5, 'linewidth': 0})
    sns.lineplot(data=PSLS_FTNC_melt_s_filtered, x='Genomic Distance', y='P(s)', 
                 errorbar="se", linewidth=1, color=color, err_kws={'alpha': 0.5, 'linewidth': 0})    

    axes.set_xscale('log')
    axes.set_yscale('log')
    axes.set_xlim((100, 1000))
    axes.set_ylim((0.001, 0.006))
    axes.set_xlabel('')
    axes.set_ylabel('')
    axes.set_title(f'Zoomed P(s) vs Genomic Distance, {chromatin_type}')

    plt.tight_layout()
    plt.savefig(f'{output_path}/PS_Segment_{condition}_{chromatin_type}_{strain}_zoom_reps.pdf', bbox_inches='tight')
    plt.close()


In [24]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/FacTether-NoCross'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results/PS_SC'

condition = base_path.split('/')[-1]

ps_segment_data_first_array_het, ps_segment_data_last_array_het, ps_segment_data_first_array_euc, ps_segment_data_last_array_euc = process_pulling_data(base_path, condition, output_path, strain)


  0%|          | 0/10 [00:00<?, ?it/s]

0.4 119000.0
0 114000.0
1 115000.0
2 116000.0
3 117000.0
4 118000.0
5 119000.0
6 120000.0
7 121000.0
8 122000.0
9 123000.0


 10%|█         | 1/10 [00:57<08:37, 57.47s/it]

10 124000.0
0.4 112000.0
0 107000.0
1 108000.0
2 109000.0
3 110000.0
4 111000.0
5 112000.0
6 113000.0
7 114000.0
8 115000.0
9 116000.0


 20%|██        | 2/10 [01:56<07:45, 58.16s/it]

...

10 107000.0
0.4 119000.0
0 114000.0
1 115000.0
2 116000.0
3 117000.0
4 118000.0
5 119000.0
6 120000.0
7 121000.0
8 122000.0
9 123000.0


 90%|█████████ | 9/10 [08:52<00:59, 59.91s/it]

10 124000.0
0.4 124000.0
0 119000.0
1 120000.0
2 121000.0
3 122000.0
4 123000.0
5 124000.0
6 125000.0
7 126000.0
8 127000.0
9 128000.0


100%|██████████| 10/10 [09:52<00:00, 59.22s/it]

10 129000.0


In [25]:
PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s = process_and_plot_ps_data(ps_segment_data_first_array_het, ps_segment_data_last_array_het, output_path, 'Heterochromatin', condition, 'navy', 'red', strain)
filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, 'Heterochromatin', condition, 'navy', 'red', strain)

PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s = process_and_plot_ps_data(ps_segment_data_first_array_euc, ps_segment_data_last_array_euc, output_path, 'Euchromatin', condition, 'navy', 'red', strain)
filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, 'Euchromatin', condition, 'navy', 'red', strain)

In [26]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/DoubleCross'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results/PS_SC'

condition = base_path.split('/')[-1]

ps_segment_data_first_array_het, ps_segment_data_last_array_het, ps_segment_data_first_array_euc, ps_segment_data_last_array_euc = process_pulling_data(base_path, condition, output_path, strain)

  0%|          | 0/10 [00:00<?, ?it/s]

0.4 148000.0
0 143000.0
1 144000.0
2 145000.0
3 146000.0
4 147000.0
5 148000.0
6 149000.0
7 150000.0
8 151000.0
9 152000.0


 10%|█         | 1/10 [00:59<08:51, 59.04s/it]

10 153000.0
0.4 126000.0
0 121000.0
1 122000.0
2 123000.0
3 124000.0
4 125000.0
5 126000.0
6 127000.0
7 128000.0
8 129000.0
9 130000.0


 20%|██        | 2/10 [02:02<08:11, 61.40s/it]

...

10 132000.0
0.4 151000.0
0 146000.0
1 147000.0
2 148000.0
3 149000.0
4 150000.0
5 151000.0
6 152000.0
7 153000.0
8 154000.0
9 155000.0


 90%|█████████ | 9/10 [09:27<01:02, 62.82s/it]

10 156000.0
0.4 149000.0
0 144000.0
1 145000.0
2 146000.0
3 147000.0
4 148000.0
5 149000.0
6 150000.0
7 151000.0
8 152000.0
9 153000.0


100%|██████████| 10/10 [10:26<00:00, 62.64s/it]

10 154000.0


In [27]:
PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s = process_and_plot_ps_data(ps_segment_data_first_array_het, ps_segment_data_last_array_het, output_path, 'Heterochromatin', condition, 'navy', 'teal', strain)
filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, 'Heterochromatin', condition, 'navy', 'teal', strain)
PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s = process_and_plot_ps_data(ps_segment_data_first_array_euc, ps_segment_data_last_array_euc, output_path, 'Euchromatin', condition, 'navy', 'teal', strain)
filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, 'Euchromatin', condition, 'navy', 'teal', strain)

In [28]:
base_path = '/Users/attar/Desktop/PS/New Folder With Items/NoTether-NoCross'

output_path = '/Users/attar/Desktop/PS/New Folder With Items/Results/PS_SC'

condition = base_path.split('/')[-1]

ps_segment_data_first_array_het, ps_segment_data_last_array_het, ps_segment_data_first_array_euc, ps_segment_data_last_array_euc = process_pulling_data(base_path, condition, output_path, strain)


  0%|          | 0/10 [00:00<?, ?it/s]

0.4 90000.0
0 85000.0
1 86000.0
2 87000.0
3 88000.0
4 89000.0
5 90000.0
6 91000.0
7 92000.0
8 93000.0
9 94000.0


 10%|█         | 1/10 [00:59<08:55, 59.47s/it]

10 95000.0
0.4 76000.0
0 71000.0
1 72000.0
2 73000.0
3 74000.0
4 75000.0
5 76000.0
6 77000.0
7 78000.0
8 79000.0
9 80000.0


 20%|██        | 2/10 [01:54<07:37, 57.14s/it]

...

10 79000.0
0.4 79000.0
0 74000.0
1 75000.0
2 76000.0
3 77000.0
4 78000.0
5 79000.0
6 80000.0
7 81000.0
8 82000.0
9 83000.0


 90%|█████████ | 9/10 [08:36<00:57, 57.25s/it]

10 84000.0
0.4 74000.0
0 69000.0
1 70000.0
2 71000.0
3 72000.0
4 73000.0
5 74000.0
6 75000.0
7 76000.0
8 77000.0
9 78000.0


100%|██████████| 10/10 [09:34<00:00, 57.48s/it]

10 79000.0


In [29]:

PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s = process_and_plot_ps_data(ps_segment_data_first_array_het, ps_segment_data_last_array_het, output_path, 'Heterochromatin', condition, 'navy', 'gray', strain)
filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, 'Heterochromatin', condition, 'navy', 'gray', strain)

PSFS_FTNC_s, PSLS_FTNC_s, PSFS_FTNC_melt_s, PSLS_FTNC_melt_s = process_and_plot_ps_data(ps_segment_data_first_array_euc, ps_segment_data_last_array_euc, output_path, 'Euchromatin', condition, 'navy', 'gray', strain)
filter_and_plot_zoomed_ps_data(PSFS_FTNC_melt_s, PSLS_FTNC_melt_s, base_path, output_path, 'Euchromatin', condition, 'navy', 'gray', strain)
